# Demo Notebook to trace Sentence Transformers model

#### [Download notebook](https://github.com/opensearch-project/opensearch-py-ml/blob/main/docs/source/examples/demo_tracing_model_torchscript_onnx.ipynb)

This notebook provides a walkthrough guidance for users to trace models from Sentence Transformers in torchScript and onnx format. After tracing the model, customers can register the model to opensearch and generate embeddings.

Remember, tracing model in torchScript or Onnx format at just two different options. We don't need to trace model in both ways. Here in our notebook we just want to show both ways. 

Step 0: Import packages and set up client

Step 1: Save model in torchScript format

Step 2: Register the saved torchScript model in Opensearch

[The following steps are optional, just showing registering model in both ways and comparing the both embedding output]

Step 3: Save model in Onnx format 

Step 4: Register the saved Onnx model in Opensearch

Step 5: Generate Sentence Embedding with registered models




## Step 0: Import packages and set up client
Install required packages for opensearch_py_ml.sentence_transformer_model
Install `opensearchpy` and `opensearch-py-ml` through pypi


In [1]:
#!pip install opensearch-py opensearch-py-ml

import os
import sys
sys.path.append(os.path.abspath(os.path.join('../../..')))

In [2]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", message="Unverified HTTPS request")
warnings.filterwarnings("ignore", message="TracerWarning: torch.tensor")
warnings.filterwarnings("ignore", message="using SSL with verify_certs=False is insecure.")

import opensearch_py_ml as oml
from opensearchpy import OpenSearch
from opensearch_py_ml.ml_models import SentenceTransformerModel
# import mlcommon to later register the model to OpenSearch Cluster
from opensearch_py_ml.ml_commons import MLCommonClient

In [3]:
CLUSTER_URL = 'https://localhost:9200'

In [4]:
def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client 

In [5]:
client = get_os_client()

# Connect to ml_common client with OpenSearch client
ml_client = MLCommonClient(client)

/home/linuxbrew/.linuxbrew/opt/python@3.8/lib/python3.8/site-packages/opensearchpy/connection/http_urllib3.py:199: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


## Step 1: Save model in torchScript format

`Opensearch-py-ml` plugin provides method `save_as_pt` which will trace a model in torchScript format and save the model in a zip file in your filesystem. 

Detailed documentation: https://opensearch-project.github.io/opensearch-py-ml/reference/api/sentence_transformer.save_as_pt.html#opensearch_py_ml.ml_models.SentenceTransformerModel.save_as_pt


Users need to provide a model id from sentence transformers (an example: `sentence-transformers/msmarco-distilbert-base-tas-b`). This model id is a huggingface model id. Exaample: https://huggingface.co/sentence-transformers/msmarco-distilbert-base-tas-b

`save_as_pt` will download the model in filesystem and then trace the model with the given input strings.

To get more direction about dummy input string please check this url: https://huggingface.co/docs/transformers/torchscript#dummy-inputs-and-standard-lengths

after tracing the model (a .pt file will be generated), `save_as_pt` method zips `tokenizers.json` and torchScript (`.pt`) file and saves in the file system. 

User can register that model to opensearch to generate embedding.

In [3]:
pre_trained_model = SentenceTransformerModel(model_id="sentence-transformers/multi-qa-MiniLM-L6-cos-v1", folder_path='sentence-transformers-torchscript/msmarco-distilbert-base-tas-b', overwrite=True)
model_path = pre_trained_model.save_as_pt(model_id="sentence-transformers/multi-qa-MiniLM-L6-cos-v1", sentences=["for example providing a small sentence", "we can add multiple sentences"])

model file is saved to  sentence-transformers-torchscript/msmarco-distilbert-base-tas-b/multi-qa-MiniLM-L6-cos-v1.pt
zip file is saved to  sentence-transformers-torchscript/msmarco-distilbert-base-tas-b/multi-qa-MiniLM-L6-cos-v1.zip 



In [6]:
pre_trained_model.make_model_config_json()

ml-commons_model_config.json file is saved at :  sentence-transformers-onxx/msmarco-distilbert-base-tas-b/ml-commons_model_config.json


'sentence-transformers-onxx/msmarco-distilbert-base-tas-b/ml-commons_model_config.json'

## Step 2: Register the saved torchScript model in Opensearch

In the last step we saved a sentence transformer model in torchScript format. Now we will register that model in opensearch cluster. To do that we can take help of `register_model` method in `opensearch-py-ml` plugin.

To register model, we need the zip file we just saved in the last step and a model config file. Example of Model config file content can be:

{
  "name": "sentence-transformers/msmarco-distilbert-base-tas-b",
  "version": "1.0.0",
  "description": "This is a port of the DistilBert TAS-B Model to sentence-transformers model: It maps sentences & paragraphs to a 768 dimensional dense vector space and is optimized for the task of semantic search.",
  "model_format": "TORCH_SCRIPT",
  "model_config": {
    "model_type": "distilbert",
    "embedding_dimension": 768,
    "framework_type": "sentence_transformers"
  }
}


`model_format` needs to be `TORCH_SCRIPT` so that internal system will look for the corresponding `.pt` file from the zip folder. 

Please refer to this doc: https://github.com/opensearch-project/ml-commons/blob/2.x/docs/model_serving_framework/text_embedding_model_examples.md


Documentation for the method: https://opensearch-project.github.io/opensearch-py-ml/reference/api/ml_commons_register_api.html#opensearch_py_ml.ml_commons.MLCommonClient.register_model

Related demo notebook about ml-commons plugin integration: https://opensearch-project.github.io/opensearch-py-ml/examples/demo_ml_commons_integration.html



In [7]:
model_config_path_torch = 'sentence-transformers-config/torchscript-config/msmarco-distilbert-base-tas-b.json'
ml_client.register_model(model_path, model_config_path_torch, isVerbose=True)

Total number of chunks 27
Sha1 value of the model file:  8c178f990a03dd66921f48da40ed81d7d4d3fc170e1bb56e1f365209e9f25027
Model meta data was created successfully. Model Id:  entcCYkB1RB1pQNOffJP
uploading chunk 1 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 2 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 3 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 4 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 5 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 6 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 7 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 8 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 9 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 10 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 11 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 12 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 13 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 14 of 27
Model id: {'status': 'Uploaded'}
u

'entcCYkB1RB1pQNOffJP'

## Step 3: Save model in Onnx format

`Opensearch-py-ml` plugin provides method `save_as_onnx` which will trace a model in ONNX format and save the model in a zip file in your filesystem. 

Detailed documentation: https://opensearch-project.github.io/opensearch-py-ml/reference/api/sentence_transformer.save_as_onnx.html#opensearch_py_ml.ml_models.SentenceTransformerModel.save_as_onnx


Users need to provide a model id from sentence transformers (an example: `sentence-transformers/msmarco-distilbert-base-tas-b`). `save_as_onnx` will download the model in filesystem and then trace the model.

after tracing the model (a .onnx file will be generated), `save_as_onnx` method zips `tokenizers.json` and torchScript (`.onnx`) file and saves in the file system. 

User can register that model to opensearch to generate embedding.


In [5]:
pre_trained_model = SentenceTransformerModel(model_id="sentence-transformers/paraphrase-MiniLM-L3-v2", folder_path='sentence-transformers-onxx/msmarco-distilbert-base-tas-b', overwrite=True)
model_path_onnx = pre_trained_model.save_as_onnx(model_id="sentence-transformers/paraphrase-MiniLM-L3-v2")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ONNX opset version set to: 15
Loading pipeline (model: sentence-transformers/paraphrase-MiniLM-L3-v2, tokenizer: sentence-transformers/paraphrase-MiniLM-L3-v2)


Creating folder sentence-transformers-onxx/msmarco-distilbert-base-tas-b/onnx
Using framework PyTorch: 2.0.1
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Found output output_1 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']
================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

zip file is saved to  sentence-transformers-onxx/msmarco-distilbert-base-tas-b/paraphrase-MiniLM-L3-v2.zip 



## Step 4: Register the saved Onnx model in Opensearch

In the last step we saved a sentence transformer model in ONNX format. Now we will register that model in opensearch cluster. To do that we can take help of `register_model` method in `opensearch-py-ml` plugin.

To register model, we need the zip file we just saved in the last step and a model config file. Example of Model config file content can be:

{
  "name": "sentence-transformers/msmarco-distilbert-base-tas-b",
  "version": "1.0.0",
  "description": "This is a port of the DistilBert TAS-B Model to sentence-transformers model: It maps sentences & paragraphs to a 768 dimensional dense vector space and is optimized for the task of semantic search.",
  "model_format": "ONNX",
  "model_config": {
    "model_type": "distilbert",
    "embedding_dimension": 768,
    "framework_type": "sentence_transformers",
    "pooling_mode":"cls",
    "normalize_result":"false"
  }
}

`model_format` needs to be `ONNX` so that internal system will look for the corresponding `.onnx` file from the zip folder.

Please refer to this doc: https://github.com/opensearch-project/ml-commons/blob/2.x/docs/model_serving_framework/text_embedding_model_examples.md


Documentation for the method: https://opensearch-project.github.io/opensearch-py-ml/reference/api/ml_commons_register_api.html#opensearch_py_ml.ml_commons.MLCommonClient.register_model

Related demo notebook about ml-commons plugin integration: https://opensearch-project.github.io/opensearch-py-ml/examples/demo_ml_commons_integration.html

In [9]:
model_config_path_onnx = 'sentence-transformers-config/onxx-config/msmarco-distilbert-base-tas-b.json'
ml_client.register_model(model_path_onnx, model_config_path_onnx, isVerbose=True)

Total number of chunks 27
Sha1 value of the model file:  311aea6defb713d46f692c18207aa7e993d19122d2331a8086445a74ac2ab031
Model meta data was created successfully. Model Id:  fHtdCYkB1RB1pQNOS_LY
uploading chunk 1 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 2 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 3 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 4 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 5 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 6 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 7 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 8 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 9 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 10 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 11 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 12 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 13 of 27
Model id: {'status': 'Uploaded'}
uploading chunk 14 of 27
Model id: {'status': 'Uploaded'}
u

'fHtdCYkB1RB1pQNOS_LY'

## Step 5: Generate Sentence Embedding

Now after loading these models in memory, we can generate embedding for sentences. We can provide a list of sentences to get a list of embedding for the sentences. 

In [10]:
# Now using this model we can generate sentence embedding.

import numpy as np

input_sentences = ["first sentence", "second sentence"]

# Generated embedding from torchScript

embedding_output_torch = ml_client.generate_embedding("entcCYkB1RB1pQNOffJP", input_sentences)

#just taking embedding for the first sentence
data_torch = embedding_output_torch["inference_results"][0]["output"][0]["data"]

# Generated embedding from onnx

embedding_output_onnx = ml_client.generate_embedding("fHtdCYkB1RB1pQNOS_LY", input_sentences)

# Just taking embedding for the first sentence
data_onnx = embedding_output_onnx["inference_results"][0]["output"][0]["data"]

# Now we can check if there's any significant difference between two outputs

print(np.testing.assert_allclose(data_torch, data_onnx, rtol=1e-03, atol=1e-05))

None
